In [ ]:
import sys
import os
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-image
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install h5py
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tables
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install Glymur
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install opencv-python

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (20.1.1)


In [ ]:
def send_notification():
    os.system("printf '\7'") 
    return None

In [ ]:
import h5py
import pandas as pd
import numpy as np
filename = "P4_catalog_v1.0_raw_classifications.hdf"
# df = pd.read_hdf(filename)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_yen, threshold_triangle, threshold_otsu, threshold_li, sobel
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from scipy import ndimage as ndi
from skimage import exposure
from skimage.io import imread

def plotter(image_url: str, advanced = False):   
    raw = imread(image_url)[..., 0]
    image = raw
    
    p2, p98 = np.percentile(image, (2, 98))
    rescaled = exposure.rescale_intensity(image, in_range=(p2, p98))
    sigma = 1
    thresh = np.mean(rescaled) -  sigma * np.std(rescaled) 
    
    bw = closing(rescaled < thresh, square(1))

    # Added post processing
    temp = ndi.binary_fill_holes(bw)
    label_objects, nb_labels = ndi.label(temp)
    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > 100
    mask_sizes[0] = 0
    clean_temp = mask_sizes[label_objects]
    
    # label image regions
    label_image = label(clean_temp)
    # to make the background transparent, pass the value of `bg_label`,
    # and leave `bg_color` as `None` and `kind` as `overlay`
    image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

    fig, ax = plt.subplots(ncols=2, figsize=(8*2, 2.5*2))
    ax[0] = plt.subplot(1, 2, 1)
    ax[1] = plt.subplot(1, 2, 2)
    ax[0].imshow(raw, cmap=plt.cm.gray)
    ax[1].imshow(image_label_overlay)
#     ax[1].imshow(clean_temp)
    

    for region in regionprops(label_image):
        # take regions with large enough areas
        if 210000 >= region.area >= 10:
            # draw rectangle around segmented coins
            
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='red', linewidth=2)
            ax[0].add_patch(rect)

    # ax[0].set_axis_off()
    ax[0].set_title(f"Raw {image_url}")
    ax[1].set_title(f"Processed")
    plt.tight_layout()
    
    if advanced:
        fig, axes = plt.subplots(ncols=2, figsize=(8*2, 2.5*2))
        ax = axes.ravel()
        ax[1] = plt.subplot(1, 2, 1)
        ax[0] = plt.subplot(1, 2, 2, sharex=ax[0], sharey=ax[0])

        ax[0].hist(rescaled.ravel(), bins=30)
        ax[0].set_title('Intensity Histogram of Rescaled Image')
        ax[0].axvline(thresh, color='r')

        ax[1].imshow(clean_temp)
        ax[1].set_title('Label')
        ax[1].axis('off')

"""
Limitations:
- Cannot pickup very small fans
- Will assume any large black thing is a fan. This includes a dip in terrain
"""

plot1 = "https://www.planetfour.org/subjects/standard/50e742bf5e2ed212400040a3.jpg"
plot2 = "https://www.planetfour.org/subjects/standard/50e742b25e2ed21240004016.jpg"

plotter(plot1, advanced = True)
plotter(plot2, advanced = True)
plt.show()

In [ ]:
from glob import glob
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

drive_location = "/Users/Utkarsh/OneDrive - University of Toronto/SURF 2020/Code"
data_location = drive_location + '/data'
P4data_location = drive_location + "/P4 data"
glob(P4data_location + "/*.csv")

metadata = pd.read_csv(P4data_location + '/P4_catalog_v1.1_metadata.csv')
tiles_coord = pd.read_csv(P4data_location + '/P4_catalog_v1.1_tile_coords_final.csv')
fan = pd.read_csv(P4data_location + "/P4_catalog_v1.1_L1C_cut_0.5_fan.csv")
blotch = pd.read_csv(P4data_location + "/P4_catalog_v1.1_L1C_cut_0.5_blotch.csv")

print(len(tiles_coord))
print(len(fan))
print(len(blotch))

item = tiles_coord.iloc[0].obsid
print(tiles_coord.iloc[0])
print(item)

pd.set_option('display.expand_frame_repr', False)
print(tiles_coord)
print(fan)
pd.set_option('display.expand_frame_repr', True)

In [ ]:
from tqdm import tqdm
import os
import glymur
import cv2
import requests
import json
import hashlib
import random

def download_file(filename):
    savename = data_location + "/{filename}_RGB.NOMAP.JP2".format(filename=filename)
    if os.path.exists(savename):
        print("{} exists".format(savename))
        return
    components = filename.split("_")
    l = 100*int(int(components[1])/100)
    h = 100*int(1+int(components[1])/100)-1
    url="https://hirise-pds.lpl.arizona.edu/download/PDS/EXTRAS/RDR/ESP/ORB_{low:06d}_{high:06d}/{filename}/{filename}_RGB.NOMAP.JP2".format(low=l,high=h,filename=filename)
    print(url)
    myfile = requests.get(url)
    with open(savename, 'wb') as file:
      file.write(myfile.content)
      file.flush()
      file.close()
    
def load_file(filename):
    savename = data_location + "/{filename}_RGB.NOMAP.JP2".format(filename=filename)
    if not os.path.exists(savename):
        download_file(filename)
    return glymur.Jp2k(savename)


nx,ny = 840, 648

def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

def fan_mask(fan, tile):
    xc,yc = fan.image_x - (tile.x_hirise-nx//2),fan.image_y - (tile.y_hirise-ny//2)
    
    fan_s = fan.distance / (1+np.tan(np.deg2rad(fan.spread//2)))
    fan_r = fan_s * np.tan(np.deg2rad(fan.spread//2))
    circ_c_points = np.cos(np.deg2rad(np.arange(0,180,10)))
    circ_s_points = np.sin(np.deg2rad(np.arange(0,180,10)))
    xp = np.hstack([0,
          fan_s,
          fan_s+fan_r*circ_s_points,
          fan_s,
          0
         ])
    yp = np.hstack([0,
          fan_r,
          fan_r*circ_c_points,
          -fan_r,
          0
         ])
    rx,ry = np.cos(np.deg2rad(fan.angle)), np.sin(np.deg2rad(fan.angle))
    rot = np.array([[rx,-ry],[ry,rx]])
    xr,yr=np.dot(rot,np.vstack([xp,yp]))

    return (xc+xr, yc+yr)

def blotch_mask(blotch, tile):
    xc,yc = blotch.image_x - (tile.x_hirise-nx//2),blotch.image_y - (tile.y_hirise-ny//2)

    t = np.linspace(0, 2*np.pi, 22)
    xp = blotch.radius_1 * np.cos(t)
    yp = blotch.radius_2 * np.sin(t)

    rx,ry = np.cos(np.deg2rad(blotch.angle)), np.sin(np.deg2rad(blotch.angle))
    rot = np.array([[rx,-ry],[ry,rx]])
    xr,yr=np.dot(rot,np.vstack([xp,yp]))

    return (xc+xr, yc+yr)


def get_image(tiles_coord, jp, irow):
    row = tiles_coord.iloc[irow]
    sx = slice(int(row.x_hirise-nx//2),int(row.x_hirise+nx//2))
    sy = slice(int(row.y_hirise-ny//2),int(row.y_hirise+ny//2))
    im16 = np.copy(jp[sy,sx])
    ratio = np.amax(im16) / 256
    img8 = (im16 / ratio).astype('uint8')
    return img8

def show_image(tiles_coord, fan_or_blotch, jp, irow, isfan=True):
    """
    isfan: True for fan and False for blotch
    """
    row = tiles_coord.loc[irow]
    img8 = get_image(tiles_coord, jp, irow)
    cv2_imshow(img8)

    if isfan:
      myfans = fan_or_blotch[fan_or_blotch.tile_id == tiles_coord.loc[irow].tile_id]
      for ifan, fan in myfans.iterrows():
            
            x,y = fan_mask(fan, row)
            x = np.where(x<0, 1, x) 
            y = np.where(y<0, 1, y) 
            x = np.where(x>nx, nx - 1, x) 
            y = np.where(y>ny, ny - 1, y) 
            plt.plot(x,y,alpha=1.0)
            
    
#     else:
#       myblotches = fan_or_blotch[fan_or_blotch.tile_id == tiles_coord.loc[irow].tile_id]
#       print(tiles_coord.loc[irow].tile_id)
#       for iblotch, blotch in myblotches.iterrows():
#           x,y = blotch_mask(blotch, row)
#           plt.plot(x,y,alpha=1.0) # Removed since we do not need blotches, false will imply no fans shown

    return img8 # This will return an array as well

for irow in [19,100]:
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    plt.figure(figsize = (16,8))
    show_image(tiles_coord, fan, jp, irow, isfan = True)
    
# print(show_image(tiles_coord, fan, jp, 100, isfan = True)) # (NO LONGER) Gives and error

I want to perform plotter using actual files with meta data before starting the machine learning

In [ ]:
from skimage.color import rgb2gray
from skimage.measure import label

def plotter2(img, advanced = False):   

    raw = img
    image = rgb2gray(img)
    
    p2, p98 = np.percentile(image, (2, 98))
    rescaled = exposure.rescale_intensity(image, in_range=(p2, p98))
    sigma = 1
    thresh = np.mean(rescaled) -  sigma * np.std(rescaled) 
    
    bw = closing(rescaled < thresh, square(1))

    # Added post processing
    temp = ndi.binary_fill_holes(bw)
    label_objects, nb_labels = ndi.label(temp)
    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > 400
    mask_sizes[0] = 0
    clean_temp = mask_sizes[label_objects]
    
    # label image regions
    label_image = label(clean_temp)
    # to make the background transparent, pass the value of `bg_label`,
    # and leave `bg_color` as `None` and `kind` as `overlay`
    image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

    fig, ax = plt.subplots(ncols=1, nrows=1, figsize = (16,8))
    ax.imshow(image, cmap=plt.cm.gray) # Black and White
      

    for region in regionprops(label_image):
        # take regions with large enough areas
        if 210000 >= region.area >= 1000:
            # draw rectangle around segmented coins
            miny, minx, maxy, maxx = region.bbox
            if miny <= 0:
                miny = 1
            if minx <= 0:
                minx = 1
            if maxy >= ny:
                maxy = ny - 1
            if maxx >= nx:
                maxx = nx - 1
            rect = mpatches.Rectangle((minx, miny), maxx - minx, maxy - miny,
                                      fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)

    # ax[0].set_axis_off()
    ax.set_title(f"B/w Boxed")


for irow in tqdm([1,2,3,20,21,22,23,24,25,26,27,28,29,100,899,1000]):
# for irow in [7]:
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    
    plotter2(img)
    show_image(tiles_coord, fan, jp, irow, isfan = True)
    plt.tight_layout()
    

I'd like to (eventually) optimise the ratio of overlap between boxes and fan masks.

## Image Classification Flow Chart

Step 1. Extract all data from masks as images keeping metadata, this is our train (target) data
        -Consider adding data which is "not fan" as target

Step 2. Use image segmentation code to extract boxes from same images keeping metadata, this is our train (unknown data)

Step 3. Run image classification machine learning to learn fan (Use id's and dictonaries to keep track)

Step 4. Accuracy score on test data

Step 5. Find accuracy for trained model on a single test image

Step 6. Use metadata to label boxed area in step 2 with accuracy score. 

### Step 1: Extracting data from masks (fan boxes)

In [ ]:
from matplotlib.patches import Rectangle

fan = pd.read_csv(P4data_location + "/P4_catalog_v1.1_L1C_cut_0.5_fan.csv")
blotch = pd.read_csv(P4data_location + "/P4_catalog_v1.1_L1C_cut_0.5_blotch.csv")

# I want to get the corners of the rectangle and an image showing the true rectangles. 

def plot_box(tiles_coord, jp, irow):
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize = (16,8))

    row = tiles_coord.loc[irow]
    img8 = get_image(tiles_coord, jp, irow)
    cv2_imshow(img8)

    myfans = fan[fan.tile_id == tiles_coord.loc[irow].tile_id]
    for ifan, fan0 in myfans.iterrows():
        x,y = fan_mask(fan0, row)
        x = np.where(x<0, 1, x) 
        y = np.where(y<0, 1, y) 
        x = np.where(x>nx, nx - 1, x) 
        y = np.where(y>ny, ny - 1, y) 
    #     ax.plot(x,y,alpha=1.0)
        left_bottom = (min(x), min(y))
        wid = max(x) - min(x)
        h = max(y) - min(y)

        rect = Rectangle(left_bottom, width = max(x) - min(x) , height = max(y) - min(y),
                         fill=False, edgecolor='green', linewidth=2)
        ax.add_patch(rect)
    return (left_bottom, wid, h)

irow = 2
row = tiles_coord.iloc[irow]
jp = load_file(row.obsid)
plot_box(tiles_coord, jp, irow)

When extracting images there will be overlaps

### New idea (Not implimenting it here)

Step 1. Find the overlap between mask and rectangle

Step 2.  Tune parameters by optimising it

In [ ]:
# I want the function to return n images with a fan in each image. 


def extract_fan_box_marking_id(tiles_coord, jp, irow):
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    row = tiles_coord.loc[irow]
    img8 = get_image(tiles_coord, jp, irow)
    marking_id_list = []

    myfans = fan[fan.tile_id == tiles_coord.loc[irow].tile_id]
    for ifan, fan0 in myfans.iterrows():
        marking_id_list.append(myfans.marking_id[ifan])     
        
    if len(marking_id_list) == 0:
        return [None]
    return marking_id_list

def extract_fan_box(tiles_coord, jp, irow):
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    row = tiles_coord.loc[irow]
    img8 = get_image(tiles_coord, jp, irow)
    cropped = []
    myfans = fan[fan.tile_id == tiles_coord.loc[irow].tile_id]

    for ifan, fan0 in myfans.iterrows():
        x,y = fan_mask(fan0, row)
        x = np.where(x<0, 1, x) 
        y = np.where(y<0, 1, y) 
        x = np.where(x>nx, nx - 1, x) 
        y = np.where(y>ny, ny - 1, y)
        min_x = int(min(x))
        max_x = int(max(x))
        min_y = int(min(y))
        max_y = int(max(y))
        cropped.append(img8[min_y:max_y, min_x:max_x])  

    if cropped == []:
        return [None]
    else:
        if cropped[0].size == 0:
            cropped.pop(0)
        return cropped # List of img8 files of cropped fan images

for i in tqdm([1,2,5,20,29,100,899]):
    irow = i
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    marking_id_list = extract_fan_box_marking_id(tiles_coord, jp, irow)
    fan_boxes = extract_fan_box(tiles_coord, jp, irow)
    count = 0
    for image in fan_boxes:
        if image is None:
            continue
        plt.figure()
        plt.title(marking_id_list[count])    
        count += 1
        plt.imshow(image)
#         cv2_imshow(image)

In [ ]:
from numpy.random import randint
import itertools
plt.rcParams.update({'figure.max_open_warning': 0})

def intersection(arr1, arr2):
    p1 = pd.DataFrame([r.flatten() for r in arr1]).drop_duplicates()
    p2 = pd.DataFrame([r.flatten() for r in arr2]).drop_duplicates()
    res = p1.merge(p2)
    return res

def random_square_edges(threshold_size = 25):
    x1 = randint(0, nx - 1)
    y1 = randint(0, ny - 1)
    x2 = randint(0, nx - 1)
    y2 = randint(0, ny - 1) 
    
    if abs(y2-y1) < threshold_size:
#         print("random_square_edges() FAILED BOX SIZE: Trying again")
        return random_square_edges(threshold_size)

    if abs(x2-x1) < threshold_size:
#         print("random_square_edges() FAILED BOX SIZE: Trying again")
        return random_square_edges(threshold_size)
        
    if x1>x2 and y1>y2:
        x_low = x2
        x_high = x1
        y_low = y2
        y_high = y1
        
    if x1>x2 and y2>y1:
        x_low = x2
        x_high = x1
        y_low = y1
        y_high = y2
        
    if x2>x1 and y1>y2:
        x_low = x1
        x_high = x2
        y_low = y2
        y_high = y1
        
    if x2>x1 and y2>y1:
        x_low = x1
        x_high = x2
        y_low = y1
        y_high = y2
        
    return x_low, x_high, y_low, y_high

def square_generator(xmin, xmax, ymin, ymax):
    xrun = np.arange(xmin, xmax, 1)
    x = np.array([])
    y = np.array([])
    
    for xval in xrun:
        yline = np.arange(ymin, ymax, 1)
        xline = np.linspace(xval, xval, len(yline))
        x = np.append(x, xline)
        y = np.append(y, yline)
    return x,y

def extract_random_box(tiles_coord, jp, irow, check_intersection = False):
    """ This function extracts a random box which is not a fan but may be a blotch
    """
    
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    img = get_image(tiles_coord, jp, irow)
    row = tiles_coord.loc[irow]
    img8 = get_image(tiles_coord, jp, irow)

    myfans = fan[fan.tile_id == tiles_coord.loc[irow].tile_id]
    
    fansx = np.array([])
    fansy = np.array([])
    
    for ifan, fan0 in myfans.iterrows():
        x,y = fan_mask(fan0, row)
        x = np.where(x<0, 1, x) 
        y = np.where(y<0, 1, y) 
        x = np.where(x>nx, nx - 1, x) 
        y = np.where(y>ny, ny - 1, y)
        xtemp, ytemp = square_generator(min(x), max(x), min(y), max(y))
        xtemp, ytemp = np.round(xtemp), np.round(ytemp)
        fansx = np.append(fansx, xtemp)
        fansy = np.append(fansy, ytemp)
#         plt.plot(xtemp,ytemp, alpha = 0.3, color = "g")
    
    x_low, x_high, y_low, y_high = random_square_edges(threshold_size = 50)
    
    X,Y = square_generator(x_low, x_high, y_low, y_high)
    
    if check_intersection:
        arr1 = np.vstack((fansx,fansy)).T
        arr2 = np.vstack((X,Y)).T
        intersect = intersection(arr1, arr2)
        if intersect.size != 0:
#             print("extract_random_box() FAILED INTERSECTION: Trying again")
            return extract_random_box(tiles_coord, jp, irow, check_intersection = True) #If intersecting with fan masks, recurse. 
        
    maxx, minx = int(max(X)), int(min(X))
    maxy, miny = int(max(Y)), int(min(Y))
#     cv2_imshow(img8)
#     plt.plot(X,Y, alpha = 0.3, color = "r")
        
    return img8[miny:maxy, minx:maxx]


for i in tqdm([1,2,5,20,29,100,899]):
    irow = i
    row = tiles_coord.iloc[irow]
    jp = load_file(row.obsid)
    plt.figure(figsize = (16,8))
    image = extract_random_box(tiles_coord, jp, irow)
    plt.imshow(image)
    

I now need to save the data and add a format for knowing which tiles is what 

###### Goal:
Set a true/false label to files if it is a fan or not. 

Something similar to

```
data:{tile_id1: {marking_id1: {label:['fan'/'notfan'], img: ...}, 
                {marking_id2: {label:['fan'/'notfan'], img: ...}
      tile_id2: {marking_id1: {label:['fan'/'notfan'], img: ...}, 
                {marking_id2: {label:['fan'/'notfan'], img: ...}}
```

```
Dataframe Approach
|tile_id   | marking_id|label  |img   |
|APF0000ci9|F000000    |fan    | .....|
|APF0000ci9|F000001    |fan    | .....|
|APF0000ci9|R000001    |notfan | .....|
|APF0000ci9|R000002    |notfan | .....|
```

### Step 2: Splitting data while maintianing metadata
#### Structuring data

In [ ]:
data = pd.DataFrame({'tile_id' : [], "marking_id": [], "label": [], "img": []})

print("No. Tiles:", len(tiles_coord))
print("Total Sample Size:", len(fan))
for i in tqdm(range(50)):
    irow = i
    
    marking_id_list = extract_fan_box_marking_id(tiles_coord, jp, irow)
    fan_boxes = extract_fan_box(tiles_coord, jp, irow)
    
    if marking_id_list[0] is None:
        continue
    
    assert len(marking_id_list) == len(fan_boxes)
    
    for i in range(len(marking_id_list)):
        boxed_fan = pd.DataFrame({'tile_id' : [tiles_coord.iloc[irow].tile_id],
                            "marking_id": [marking_id_list[i]],
                            "label": ["fan"], 
                            "img": [fan_boxes[i]]})
        
        data = pd.concat([data, boxed_fan])
        
        random = pd.DataFrame({'tile_id' : [tiles_coord.iloc[irow].tile_id],
                            "marking_id": ["R" + marking_id_list[i][1:]],
                            "label": ["notfan"], 
                            "img": [extract_random_box(tiles_coord, load_file(tiles_coord.iloc[irow].obsid), irow)]})
        data = pd.concat([data, random])


data.reset_index(drop=True, inplace=True)
print("Current Sample Size:" , data.shape[0]//2)
data

In [ ]:
label = "notfan"
# label = "fan"
for index, row in data.loc[data["label"] == label].iterrows():
    plt.figure()
    plt.imshow(row.img)


Are we sure that we want to give it some images that clearly look like fans? It just seems a bit counterintuitive.


### Step 3: Time for Machine Learning! (Finally)

In [ ]:
send_notification()